In [ ]:
%pip install torch

In [ ]:
import torch

# 1. Simple gradient calculation with a scalar
print("=== Simple Scalar Gradient ===")
x = torch.tensor([2.0], requires_grad=True)
y = x ** 2  # Simple quadratic function
y.backward()  # Calculate gradient
print(f"dy/dx at x=2: {x.grad}")  # Should be 4 (derivative of x^2 is 2x)

# 2. Gradient with vector input
print("\n=== Vector Gradient ===")
x = torch.tensor([2.0, 3.0], requires_grad=True)
y = x.sum() ** 2
y.backward()
print(f"Gradient with respect to x: {x.grad}")

# 3. More complex function example
print("\n=== Complex Function Gradient ===")
x = torch.tensor([1.0], requires_grad=True)
y = torch.sin(x) * torch.exp(-x)
y.backward()
print(f"Gradient of sin(x)*e^(-x) at x=1: {x.grad}")

# 4. Gradient with neural network-like operations
print("\n=== Neural Network-like Operations ===")
weights = torch.tensor([1.0, 2.0], requires_grad=True)
inputs = torch.tensor([1.0, 1.0])
output = torch.sum(weights * inputs)
output.backward()
print(f"Gradient with respect to weights: {weights.grad}")